In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from tqdm.notebook import trange, tqdm
import numpy as np
import matplotlib.pyplot as plt

import copy
import random
import time

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!unzip "/content/gdrive/My Drive/cropped_image_dataset2.zip"

Archive:  /content/gdrive/My Drive/cropped_image_dataset2.zip
   creating: cropped_image_dataset2/
   creating: cropped_image_dataset2/train/
   creating: cropped_image_dataset2/train/0/
  inflating: cropped_image_dataset2/train/0/904.jpg  
  inflating: cropped_image_dataset2/train/0/915.jpg  
  inflating: cropped_image_dataset2/train/0/916.jpg  
  inflating: cropped_image_dataset2/train/0/920.jpg  
  inflating: cropped_image_dataset2/train/0/922.jpg  
  inflating: cropped_image_dataset2/train/0/925.jpg  
  inflating: cropped_image_dataset2/train/0/928.jpg  
  inflating: cropped_image_dataset2/train/0/935.jpg  
  inflating: cropped_image_dataset2/train/0/937.jpg  
  inflating: cropped_image_dataset2/train/0/943.jpg  
  inflating: cropped_image_dataset2/train/0/947.jpg  
  inflating: cropped_image_dataset2/train/0/950.jpg  
  inflating: cropped_image_dataset2/train/0/954.jpg  
  inflating: cropped_image_dataset2/train/0/958.jpg  
  inflating: cropped_image_dataset2/train/0/961.jpg  
  i

In [4]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [5]:
INPUT_DIM = 28 * 28
OUTPUT_DIM = 10
TRAIN_DATA_PATH = 'cropped_image_dataset2/train'
TEST_DATA_PATH = 'cropped_image_dataset2/test'
BATCH_SIZE = 64
EPOCHS = 20

In [6]:
train_transforms = transforms.Compose([
                            # transforms.CenterCrop(28, padding=2),
                            transforms.RandomRotation(5),
                            transforms.Resize((28,28)),
                            transforms.Grayscale(1),
                            transforms.ToTensor(),
                            # transforms.Normalize(mean=[mean], std=[std])
                                        ])
                            
test_transforms = transforms.Compose([
                            transforms.Grayscale(1),
                            transforms.Resize((28,28)),
                            transforms.ToTensor(),
                            # transforms.Normalize(mean=[mean], std=[std])
                                        ])
train_data =  datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=train_transforms)
test_data =  datasets.ImageFolder(root=TEST_DATA_PATH, transform=test_transforms)


In [7]:
train_iterator = data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_iterator = data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

In [8]:
VALID_RATIO = 0.9

n_train_examples = int(len(train_data) * VALID_RATIO)
n_valid_examples = len(train_data) - n_train_examples

In [9]:
train_data, valid_data = data.random_split(train_data,
                                           [n_train_examples, n_valid_examples])

In [10]:
valid_data = copy.deepcopy(valid_data)
valid_data.dataset.transform = test_transforms

In [11]:
BATCH_SIZE = 64

train_iterator = data.DataLoader(train_data,
                                 shuffle=True,
                                 batch_size=BATCH_SIZE)

valid_iterator = data.DataLoader(valid_data,
                                 batch_size=BATCH_SIZE)

test_iterator = data.DataLoader(test_data,
                                batch_size=BATCH_SIZE)

In [12]:
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()

        self.input_fc = nn.Linear(input_dim, 250)
        self.hidden_fc = nn.Linear(250, 100)
        self.output_fc = nn.Linear(100, output_dim)

    def forward(self, x):

        # x = [batch size, height, width]

        batch_size = x.shape[0]

        x = x.view(batch_size, -1)

        # x = [batch size, height * width]

        h_1 = F.relu(self.input_fc(x))

        # h_1 = [batch size, 250]

        h_2 = F.relu(self.hidden_fc(h_1))

        # h_2 = [batch size, 100]

        y_pred = self.output_fc(h_2)

        # y_pred = [batch size, output dim]

        return y_pred, h_2

In [13]:
class Net(nn.Module):
    #This defines the structure of the NN.
    def __init__(self):
        super(Net, self).__init__()
        # input is 28x28
        # padding=2 for same padding
        self.conv1 = nn.Conv2d(1, 32, 5, padding=2)
        # feature map size is 14*14 by pooling
        # padding=2 for same padding
        self.conv2 = nn.Conv2d(32, 64, 5, padding=2)
        # feature map size is 7*7 by pooling
        self.fc1 = nn.Linear(64*7*7, 1024)
        self.fc2 = nn.Linear(1024, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, 64*7*7)   # reshape Variable
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x), "hehe"

In [14]:
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output, "hehe"

In [15]:
# model = MLP(INPUT_DIM, OUTPUT_DIM)
model = Net()
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = criterion.to(device)

In [16]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim=True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    # print("pred:"+str(top_pred))
    # print("correct:"+str(y))

    return acc

In [17]:
def train(model, iterator, optimizer, criterion, device):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for (x, y) in tqdm(iterator, desc="Training", leave=False):

        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()

        y_pred, _ = model(x)
        # y_pred = model(x)

        # loss = criterion(y_pred, y)
        loss = F.nll_loss(y_pred, y)

        acc = calculate_accuracy(y_pred, y)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [18]:
def evaluate(model, iterator, criterion, device):

    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():

        for (x, y) in tqdm(iterator, desc="Evaluating", leave=False):

            x = x.to(device)
            y = y.to(device)

            y_pred, _ = model(x)
            # y_pred = model(x)
            # print(y_pred)
            # print(y)

            # loss = criterion(y_pred, y)
            loss = F.nll_loss(y_pred, y)

            acc = calculate_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [19]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [20]:
EPOCHS = 30

best_valid_loss = float('inf')

for epoch in trange(EPOCHS):

    start_time = time.monotonic()

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'cropped_cnn_model.pt')

    end_time = time.monotonic()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
# torch.save(model.state_dict(), 'cropped_model.pt')


  0%|          | 0/30 [00:00<?, ?it/s]

Training:   0%|          | 0/37 [00:00<?, ?it/s]

<ipython-input-13-0a3f685c77b1>:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x), "hehe"


Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 0m 9s
	Train Loss: 1.646 | Train Acc: 41.61%
	 Val. Loss: 0.907 |  Val. Acc: 70.85%


...

Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 29 | Epoch Time: 0m 2s
	Train Loss: 0.001 | Train Acc: 100.00%
	 Val. Loss: 0.005 |  Val. Acc: 99.69%


Training:   0%|          | 0/37 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 30 | Epoch Time: 0m 2s
	Train Loss: 0.002 | Train Acc: 99.96%
	 Val. Loss: 0.005 |  Val. Acc: 99.69%


In [21]:
model.load_state_dict(torch.load('cropped_cnn_model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, device)

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

<ipython-input-13-0a3f685c77b1>:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x), "hehe"


In [22]:
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.001 | Test Acc: 100.00%
